ดูว่าจังหวะตัดตอนพูดต้องตัดตอนกี่วิ split chunk

overlap text ที่ merge กันจะเคลียยังไง

โมเดลรับไฟล์เสียงสั้นสุดเท่าไร โมเดลถูกจูนมาแล้วให้รับไฟล์เสียงสั้นได้

อัดเสียงยาวๆแล้วซอยไฟล์ให้มัน overlap กับแล้วเทส

ออกแบบจัดเก็บไฟล์เสียงเป็น temp file

lib สร้าง temp file

In [90]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
from package.typhoon.inference import TyphoonASR

In [93]:
model = TyphoonASR()

🌪️ Loading Typhoon ASR model on CPU...
[NeMo I 2025-11-19 13:17:30 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 2048 tokens


[NeMo W 2025-11-19 13:17:31 nemo_logging:393] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/workspace/warit/nemo-asr/stt_th_conformer_transducer_large/prepare_data/typhoon_cleanser/20250814/Split_gg/train_data_typhoon_asr_realtime.jsonl
    sample_rate: 16000
    batch_size: 8
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 30.0
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: fully_randomized
    bucketing_batch_size: null
    
[NeMo W 2025-11-19 13:17:31 nemo_logging:393] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation d

[NeMo I 2025-11-19 13:17:31 nemo_logging:381] PADDING: 0


[NeMo W 2025-11-19 13:17:32 nemo_logging:393] d:\Git\thai_voice_transcription\.venv\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
      warnings.warn(
    


[NeMo I 2025-11-19 13:17:32 nemo_logging:381] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-11-19 13:17:32 nemo_logging:381] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}


[NeMo W 2025-11-19 13:17:32 nemo_logging:393] No conditional node support for Cuda.
    Cuda graphs with while loops are disabled, decoding speed will be slower
    Reason: CUDA is not available


[NeMo I 2025-11-19 13:17:33 nemo_logging:381] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}


[NeMo W 2025-11-19 13:17:33 nemo_logging:393] No conditional node support for Cuda.
    Cuda graphs with while loops are disabled, decoding speed will be slower
    Reason: CUDA is not available


[NeMo I 2025-11-19 13:17:33 nemo_logging:381] Model EncDecRNNTBPEModel was successfully restored from C:\Users\TDGE0049\.cache\huggingface\hub\models--scb10x--typhoon-asr-realtime\snapshots\a14b79d50c788dbdfe559c8a28a9b90153cf3865\typhoon-asr-realtime.nemo.


In [112]:
import glob

# ถอดเสียงทีละ chunk
chunk_files = sorted(glob.glob("audio/chunks_fah/*.wav"))
texts = []

for chunk_file in chunk_files:
    processed = model.preprocess(chunk_file)
    result = model.transcribe(processed)
    text = result['text'][0] if result['text'] else ''
    texts.append(text)

   Resampled: 48000 Hz → 16000 Hz
✅ Saved: processed_audio.wav (2.0s)


Transcribing: 100%|██████████| 1/1 [00:00<00:00,  5.90it/s]


   Resampled: 48000 Hz → 16000 Hz
✅ Saved: processed_audio.wav (2.0s)


Transcribing: 100%|██████████| 1/1 [00:00<00:00,  5.89it/s]


   Resampled: 48000 Hz → 16000 Hz
✅ Saved: processed_audio.wav (2.0s)


...

Transcribing: 100%|██████████| 1/1 [00:00<00:00,  3.58it/s]


   Resampled: 48000 Hz → 16000 Hz
✅ Saved: processed_audio.wav (2.0s)


Transcribing: 100%|██████████| 1/1 [00:00<00:00,  3.97it/s]


   Resampled: 48000 Hz → 16000 Hz
✅ Saved: processed_audio.wav (2.0s)


Transcribing: 100%|██████████| 1/1 [00:00<00:00,  5.62it/s]


In [96]:
def merge_overlapped_text(texts):
    if not texts:
        return ""
    
    merged = texts[0]
    
    for i in range(1, len(texts)):
        current = texts[i]
        if not current:
            continue
            
        # หาส่วนที่ซ้ำกันระหว่างท้าย merged กับต้น current
        best_overlap = 0
        for j in range(1, min(len(merged), len(current)) + 1):
            if merged[-j:] == current[:j]:
                best_overlap = j
        
        # รวมโดยตัดส่วนที่ซ้ำออก
        merged += current[best_overlap:]
    
    return merged

from rapidfuzz import fuzz

def merge_overlapped_text_fuzzy(texts, threshold=85):
    if not texts:
        return ""

    merged = texts[0]

    for i in range(1, len(texts)):
        current = texts[i]
        if not current:
            continue

        best_overlap = 0
        max_len = min(len(merged), len(current))

        for j in range(1, max_len + 1):
            tail = merged[-j:]
            head = current[:j]

            # ใช้ fuzzy ratio แทน exact match
            similarity = fuzz.ratio(tail, head)
            if similarity >= threshold:  
                best_overlap = j

        merged += current[best_overlap:]

    return merged

In [109]:
texts

['ดีนี่น้องฟ้าเองตอนนี้เวลา',
 'ตอนนี้เวลาเก้าโมงสามสิบ',
 'เก้าโมงสามสิบเจ็ดนาทีอากาศ',
 'นาทีอากาศเย็นเจี๊ยบ',
 'เดนเจี๊ยบโยมิเตอร์',
 'ยอพิเตอร์',
 '',
 'แล้วก็จอดกินฟ้า',
 'ก็จอดมีฟ้าทําเรื่องฟอร์ด',
 'ทําเรื่องปลอดใครยังอยู่ในข้อ',
 'เตรียมอยู่ในข้อฮอลลี',
 'ไม่มีไอเดียอะไรเลย',
 'ไม่มีไอเดียอะไรเลยช่วยแชร์หน่อยได้',
 'ช่วยแชร์หน่อยได้ไหม',
 'ช่วยน้องที']

In [111]:
texts

['ดีนี่น้องฟ้าเองตอนนี้เวลา',
 'เวลาเก้าโมงสามสิบเจ็ดนาที',
 'อากาศเย็นเจี๊ยบ',
 'ยอพี่เตอร์',
 'แล้วก็จอดกินฟ้า',
 'ทําเรื่องฟอร์ดเคลียร์อยู่',
 'อยู่ในข้อฮอลเลยดีไม่มีไอเดีย',
 'ไอเดียอะไรเลยช่วยแชร์หน่อยได้ไหม',
 'ช่วยน้องที']

In [113]:
texts

['ดีนี่น้องฟ้าเองตอนนี้เวลา',
 'เก้าโมงสามสิบเจ็ดนาทีอากาศ',
 'เดนเจี๊ยบโยมิเตอร์',
 '',
 'ก็จอดมีฟ้าทําเรื่องฟอร์ด',
 'เตรียมอยู่ในข้อฮอลลี',
 'ไม่มีไอเดียอะไรเลยช่วยแชร์หน่อยได้',
 'ช่วยน้องที']

In [103]:
# รวมข้อความ
final_text = merge_overlapped_text(texts)
print(f"\noutput: {final_text}")

final_text = merge_overlapped_text_fuzzy(texts=texts, threshold=85)
print(f"\noutput: {final_text}")


output: ดีนี่น้องฟ้าเองตอนนี้เวลาเก้าโมงสามสิบเจ็ดนาทีอากาศเย็นเจี๊ยบโยมิเตอร์แล้วก็จัดหมีฟ้าทําเรื่อยทําเรื่องปลอดใครยังอยู่ในข้อฮอลเลยดีไม่มีไอเดียอะไรเลยช่วยแชร์หน่อยได้ไหมช่วยน้องทีร้องที

output: ดีนี่น้องฟ้าเองตอนนี้เวลาก้าโมงสามสิบเจ็ดนาทีอากาศเย็นเจี๊ยบโยมิเตอร์แล้วก็จัดหมีฟ้าทําเรื่อยปลอดใครยังอยู่ในข้ออลเลยดีไม่มีไอเดียะไรเลยช่วยแชร์หน่อยได้มช่วยน้องทีร้องที


เทสว่า chunk กี่วิ overlap กี่วิอันไหนเร็วกว่ากัน ในตอนทำ realtime

In [81]:
# from pythainlp.tokenize import word_tokenize

# def tokenize(text):
#     return word_tokenize(text)


# def align_sequences(seq1, seq2):
#     n, m = len(seq1), len(seq2)

#     dp = [[0] * (m + 1) for _ in range(n + 1)]
#     bt = [[None] * (m + 1) for _ in range(n + 1)]

#     for i in range(1, n + 1):
#         dp[i][0] = i
#         bt[i][0] = (i - 1, 0)
#     for j in range(1, m + 1):
#         dp[0][j] = j
#         bt[0][j] = (0, j - 1)

#     for i in range(1, n + 1):
#         for j in range(1, m + 1):
#             cost = 0 if seq1[i-1] == seq2[j-1] else 1

#             choices = [
#                 (dp[i-1][j] + 1, (i-1, j)),
#                 (dp[i][j-1] + 1, (i, j-1)),
#                 (dp[i-1][j-1] + cost, (i-1, j-1))
#             ]
#             dp[i][j], bt[i][j] = min(choices, key=lambda x: x[0])

#     aligned = []
#     i, j = n, m
#     while i > 0 or j > 0:
#         pi, pj = bt[i][j]
#         if pi == i - 1 and pj == j - 1:
#             aligned.append((i-1, j-1))
#         elif pi == i - 1 and pj == j:
#             aligned.append((i-1, None))
#         else:
#             aligned.append((None, j-1))
#         i, j = pi, pj

#     aligned.reverse()
#     return aligned


# def merge_two(prev_text, new_text):
#     seq1 = tokenize(prev_text)
#     seq2 = tokenize(new_text)

#     print(seq1)
#     print(seq2)
#     alignment = align_sequences(seq1, seq2)
#     print(alignment)
#     cut_pos = 0
#     for (i, j) in alignment:
#         if i is None or j is None:
#             cut_pos = j if j is not None else cut_pos
#             break
#         if seq1[i] != seq2[j]:
#             cut_pos = j
#             break
#         cut_pos = j + 1

#     return prev_text + new_text[len(''.join(seq2[:cut_pos])):]


# def merge_overlapped_text_aligner(texts):
#     if not texts:
#         return ""

#     merged = texts[0]
#     for i in range(1, len(texts)):
#         merged = merge_two(merged, texts[i])
#         print(merged)

#     return merged

# def merge_two(prev_text, new_text):
#     seq1 = tokenize(prev_text)
#     seq2 = tokenize(new_text)
#     print(seq1)
#     print(seq2)
#     alignment = align_sequences(seq1, seq2)

#     # หาตำแหน่ง token ใหม่ใน seq2
#     # หลักการ: new tokens คือคู่ที่ i == None
#     new_tokens = []
#     for (i, j) in alignment:
#         if i is None and j is not None:
#             new_tokens.append(seq2[j])

#     # merge = seq1 + new_tokens
#     merged_tokens = seq1 + new_tokens
#     return ''.join(merged_tokens)


In [82]:
# merge_overlapped_text_aligner(texts)